In [1]:
!nvidia-smi

Wed Jul  8 00:00:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
!pip install -U PyYAML

     |████████████████████████████████| 276kB 8.5MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=fff28afd6d58bf6a4aefc5225a58e42a0d19db9bc7d84b6402ce53a8a9e15508
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [5]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 7335 (delta 40), reused 39 (delta 19), pack-reused 7255
Receiving objects: 100% (7335/7335), 13.89 MiB | 13.01 MiB/s, done.
Resolving deltas: 100% (4940/4940), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-920wtyxi
Created temporary directory: /tmp/pip-req-tracker-se07w25z
Created requirements tracker '/tmp/pip-req-tracker-se07w25z'
Created temporary directory: /tmp/pip-install-5ykebwvj
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-onukj9q_
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-se07w25z'
    Running setup.py (path

In [6]:
import torch.random
import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
zip_name = 'gwd-split-0.7.zip'
zip_path = '/content/drive/My Drive/Colab Notebooks/gwdsplit/' + zip_name
!cp "{zip_path}" .
!unzip -q '{zip_name}'
!rm '{zip_name}'

In [8]:
weight_name = 'yolov5x_coco.pt'
weight_path = '/content/drive/My Drive/Colab Notebooks/yolov5weights/' + weight_name
!cp '{weight_path}' .

In [9]:
train = '/content/drive/My Drive/Colab Notebooks/yolov5/train.py'
data = '/content/drive/My Drive/Colab Notebooks/yolov5config/wheat_colab.yaml'
cfg = '/content/drive/My Drive/Colab Notebooks/yolov5config/yolov5x.yaml'
weights = '/content/' + weight_name
name = 'x-b4-e25-coco'

In [10]:
!python '{train}' --img 1024 --batch 4 --epochs 25 --data '{data}' --cfg '{cfg}' --weights '{weights}' --name '{name}'

{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=4, bucket='', cache_images=False, cfg='/content/drive/My Drive/Colab Notebooks/yolov5config/yolov5x.yaml', data='/content/drive/My Drive/Colab Notebooks/yolov5config/wheat_colab.yaml', device='', epochs=25, evolve=False, img_size=[1024], multi_scale=False, name='x-b4-e25-coco', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='/content/yolov5x_coco.pt')
Using CUDA Apex device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-07-08 00:08:58.863026: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Start Tensorboard with "tensorboar

In [11]:
import shutil
new_weight = '/content/drive/My Drive/Colab Notebooks/yolov5weights/' + name
if os.path.exists(new_weight):
  shutil.rmtree(new_weight)
os.mkdir(new_weight)

weight_last = '/content/weights/last_' + name + '.pt'
weight_best = '/content/weights/best_' + name + '.pt'

!cp '{weight_last}' '{new_weight}'
!cp '{weight_best}' '{new_weight}'
!cp 'results.png' '{new_weight}'
!cp 'labels.png' '{new_weight}'
!cp 'test_batch0_gt.jpg' '{new_weight}'
!cp 'test_batch0_pred.jpg' '{new_weight}'